In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import os

In [2]:
path_Productos='Bases de datos/Productos.csv'
path_Ventas='Bases de datos/Ventas.csv'
path_Despachos='Bases de datos/Despachos.csv'
path_Lista_Ventas = "Bases de datos/ListaVentas.pkl"
path_Ofertas="Bases de datos/Ofertas.pkl"

In [3]:
try:
    Productos=pd.read_csv(path_Productos, encoding='utf-8')
    Productos.set_index('ID_PROD',inplace=True)
except:
    print('No se encontro base de datos de Productos.')
    Productos=pd.DataFrame(columns=['PRODUCTO','ID_PROD','DESCRIPCIÓN','CANTIDAD_DISPONIBLE','PRECIO'])
    Productos.set_index('ID_PROD',inplace=True)
    Productos.to_csv(path_Productos, encoding='utf-8')
try:
    Ventas=pd.read_csv(path_Ventas, encoding='utf-8')
    Ventas.set_index('ID_VENTA',inplace=True)
except:
    print('No se encontro base de datos de Ventas.')
    Ventas=pd.DataFrame(columns=['ID_VENTA','FECHA','MONTO_PRODUTOS','ID_DESPACHO','MONTO_TOTAL','ID_COMPRADOR(RUT)',
                                 'TELEFONO',
                                'DESCUENTO'])
    Ventas.set_index('ID_VENTA',inplace=True)
    Ventas.to_csv(path_Ventas, encoding='utf-8')
try:
    Despachos=pd.read_csv(path_Despachos, encoding='utf-8')
    Despachos.set_index('ID_DESPACHO',inplace=True)
except:
    print('No se encontro base de datos de Despachos.')
    Despachos=pd.DataFrame(columns=['ID_DESPACHO','FECHA','ID_VENTA','CALLE','NUMERO','DEPTO-OTROS','COMUNA','FECHA_ENTREGA',
                                   'ESTADO'])
    Despachos.set_index('ID_DESPACHO',inplace=True)
    Despachos.to_csv(path_Despachos, encoding='utf-8')
    
if os.path.exists(path_Lista_Ventas):
    with open(path_Lista_Ventas, 'rb') as file:
        Lista_Ventas=pickle.load(file)
else:
    print('Listas de ventas no existen.')
    with open(path_Lista_Ventas, 'wb') as file:
        Lista_Ventas={}
        pickle.dump(Lista_Ventas, file)
if os.path.exists(path_Ofertas):
    with open(path_Ofertas, 'rb') as file:
        Ofertas=pickle.load(file)
else:
    print('Listas de ofertas no existen.')
    with open(path_Ofertas, 'wb') as file:
        Ofertas={}
        pickle.dump(Ofertas, file)

In [4]:
### Diccionario que guarda los productos comprados en cada venta.

def crear_oferta(nombre_oferta,dic_productos,precio):
    """
    nombre_oferta:  Nombre de la oferta.
    dic_productos: Diccionario que tiene como clave el nombre de los productos de la oferta y como valor su cantidad.
    precio: Precio de la oferta.
    """
    for prod in dic_productos:
        if prod not in Productos['PRODUCTO'].values:
            print('Producto: '+prod+' no existe.')
    Ofertas[nombre_oferta]=dic_productos.copy()
    with open(path_Ofertas, 'wb') as file:
        pickle.dump(Ofertas, file)
    ####Agregamos la oferta como producto:
    descripcion='Oferta: '

    for prod in dic_productos:
        descripcion=descripcion+prod+'('+str(dic_productos[prod])+')'+' '
    cantidad=np.infty
    for prod in dic_productos:
        cantidad_aux=Productos.loc[Productos['PRODUCTO']==prod,'CANTIDAD_DISPONIBLE'].values[0]
        cantidad=min(cantidad,cantidad_aux //dic_productos[prod])
    
    crear_producto(nombre_oferta,descripcion,cantidad,precio)
def crear_producto(producto,descripcion,cantidad,precio):
    """
    producto: Nombre del producto.
    descripcion: Descripción del producto.
    cantidad: Cantidad inicial del producto.
    precio: Precio inicial del producto.
    """
    ###Verificar si el producto existe:
    if producto in Productos['PRODUCTO'].values:
        print('Producto existente.')
        return 
    id_prod_new=len(Productos)
    Productos.loc[id_prod_new,'PRODUCTO']=producto
    Productos.loc[id_prod_new,'DESCRIPCIÓN']=descripcion
    Productos.loc[id_prod_new,'CANTIDAD_DISPONIBLE']=cantidad
    Productos.loc[id_prod_new,'PRECIO']=precio
    Productos.to_csv(path_Productos, encoding='utf-8')
def modificar_producto(id_producto,cantidad=None,precio=None):
    """
    id_producto: ID del producto.
    cantidad: Nueva cantidad.
    precio: Nuevo precio.
    """
    if id_producto not in Productos.index.unique():
        print('Indice de producto no válido.')
        return 
    if cantidad is None and precio is None:
        print('Debe ingresar una nueva cantidad o precio.')
        return 
    if precio is not None:
        Productos.loc[id_producto,'PRECIO']=int(precio)
    if cantidad is not None:
        Productos.loc[id_producto,'CANTIDAD_DISPONIBLE']=int(cantidad)
    print('Producto actualziado exitosamente.')
    Productos.to_csv(path_Productos, encoding='utf-8')
def crear_despacho(id_venta,fecha,calle,numero,depto,comuna,fecha_entrega,monto_despacho):
    """
    id_venta: Identificador de la venta asociada.
    fecha: Fecha de generación del despacho.
    calle: Nombre de la calle.
    numero: Numero de la dirección.
    depto: Numero del depto.
    comuna: Nombre de la comuna.
    fecha_entrega: Fecha inicial de la entrega.
    
    Función que crea un nuevo despacho. Genera el ID_DESPACHO y lo agrega a la custionada.
    
    """
    id_despacho_new=len(Despachos)+2100
    Despachos.loc[id_despacho_new,'FECHA']=fecha
    Despachos.loc[id_despacho_new,'FECHA_ENTREGA']=fecha_entrega
    Despachos.loc[id_despacho_new,'COMUNA']=comuna
    Despachos.loc[id_despacho_new,'DEPTO-OTROS']=depto
    Despachos.loc[id_despacho_new,'NUMERO']=numero
    Despachos.loc[id_despacho_new,'CALLE']=calle
    Despachos.loc[id_despacho_new,'ID_VENTA']=id_venta
    Despachos.to_csv(path_Despachos, encoding='utf-8')
    return id_despacho_new
def crear_venta(id_comprador,telefono,lista_productos,fecha,calle=None,numero=None,depto=None,comuna=None,
                fecha_entrega=None,
               monto_despacho=None,descuento=0):
    """
    id_comprador: Identificador comprado
    lista_productos: diccionario con los ID y las contidades de los productos comprados.
    fecha: Fecha de generación de la venta.
    
    Las siguientes variables son para crear el despacho.
    
    calle: Nombre de la calle.
    numero: Numero de la dirección.
    depto: Numero del depto.
    comuna: Nombre de la comuna.
    fecha_entrega: Fecha inicial de la entrega.
    
    Función que crea una nueva venta. Genera el ID_VENTA y agrega la venta a la tabla de ventas.
    """
    
    id_venta_new=len(Ventas)+2000
    Lista_Ventas[id_venta_new]=lista_productos.copy()
    monto_prods=0
    for id_prod in Lista_Ventas[id_venta_new]:
        ###Actualizamos las cantidades
        monto_prods=monto_prods+Lista_Ventas[id_venta_new][id_prod]*int(Productos.loc[id_prod,'PRECIO'])
        if Productos.loc[id_prod,'PRODUCTO'] in Ofertas:
            for prod in Ofertas[Productos.loc[id_prod,'PRODUCTO']]:
                ###Obtenemos la clave del producto
                id_prod_aux=Productos.loc[Productos['PRODUCTO']==prod].index.values
                ##Actualizamos la cantidad de cada producto en la oferta
                Productos.loc[id_prod_aux,'CANTIDAD_DISPONIBLE']=(Productos.loc[id_prod_aux,'CANTIDAD_DISPONIBLE']
                -Ofertas[Productos.loc[id_prod,'PRODUCTO']][prod]*Lista_Ventas[id_venta_new][id_prod])
        else:
            Productos.loc[id_prod,'CANTIDAD_DISPONIBLE']=(int(Productos.loc[id_prod,'CANTIDAD_DISPONIBLE'])-
                                                         Lista_Ventas[id_venta_new][id_prod])
    ###Actualizamos la cantidad de las ofertas.
    for id_prod in Lista_Ventas[id_venta_new]:    
        if Productos.loc[id_prod,'PRODUCTO'] in Ofertas:
            cantidad_aux=np.infty
            for prod in Ofertas[Productos.loc[id_prod,'PRODUCTO']]:
                cantidad_aux=Productos.loc[Productos['PRODUCTO']==prod,'CANTIDAD_DISPONIBLE'].values[0]
                cantidad=min(cantidad,cantidad_aux //dic_productos[prod])
            Productos.loc[id_prod,'CANTIDAD_DISPONIBLE']=cantidad
    
    monto_prods=np.sum([ Lista_Ventas[id_venta_new][id_prod]*int(Productos.loc[id_prod,'PRECIO']) 
                   for id_prod in Lista_Ventas[id_venta_new]])
    id_despacho=crear_despacho(id_venta_new,fecha,calle,numero,depto,comuna,fecha_entrega,monto_despacho)
    Ventas.loc[id_venta_new,'FECHA']=fecha
    Ventas.loc[id_venta_new,'MONTO_PRODUTOS']=monto_prods
    Ventas.loc[id_venta_new,'MONTO_TOTAL']=monto_prods+monto_despacho-descuento
    Ventas.loc[id_venta_new,'DESCUENTO']=descuento
    Ventas.loc[id_venta_new,'ID_DESPACHO']=id_despacho
    Ventas.loc[id_venta_new,'ID_COMPRADOR(RUT)']=id_comprador
    Ventas.loc[id_venta_new,'TELEFONO']=telefono
    Ventas.to_csv(path_Ventas, encoding='utf-8')
    with open(path_Lista_Ventas, 'wb') as file:
        pickle.dump(Lista_Ventas, file)
    return id_venta_new

In [5]:
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter import messagebox
from datetime import datetime
from tkinter import filedialog
import functools
from pandastable import Table, TableModel
class Programa:
    def __init__(self):
        
        self.root = Tk()###El padre

        self.root.title('Super Panda Admin.')
        self.root.geometry('500x300')
        self.root.configure(background='black')

        
        self.text_archivo=StringVar()
        self.text_archivo.set('Seleccione una operación')
        self.label_archivo=Label(self.root, textvariable=self.text_archivo,justify=CENTER,bg='blue',fg='white')
        self.label_archivo.pack(fill='x',expand=True,padx=20,pady=5)
        
        self.boton_nueva_venta = Button(self.root, text='Nueva Venta', 
                                        command=self.nueva_venta, justify=CENTER)
        self.boton_nueva_venta.pack(fill='x',expand=True,padx=20,pady=5)
        
        self.boton_eliminar_venta = Button(self.root, text='Eliminar Venta', 
                                        command=self.eliminar_venta, justify=CENTER)
        
        self.boton_eliminar_venta.pack(fill='x',expand=True,padx=20,pady=5)
        
        self.boton_gestionar_prods = Button(self.root,text='Gestionar producto',command=self.gestionar_prods,
                                            justify=CENTER)
        self.boton_gestionar_prods.pack(fill='x',expand=True,padx=20,pady=5)
        
        self.boton_ver_tablas=Button(self.root,text='Ver Tablas',command=self.ver_tablas,justify=CENTER)
        self.boton_ver_tablas.pack(fill='x',expand=True,padx=20,pady=5)
        
        
        self.boton_ver_detalle_venta=Button(self.root,text='Ver Detalle Venta',command=self.ver_detalle_venta,
                                           justify=CENTER)
        self.boton_ver_detalle_venta.pack(fill='x',expand=True,padx=20,pady=5)
        
        self.boton_crear_guia_despacho=Button(self.root,text='Guía despachos',command=self.crear_guia_despacho,
                                             justify=CENTER)
        self.boton_crear_guia_despacho.pack(fill='x',expand=True,padx=20,pady=5)
        
        self.boton_crear_reporte= Button(self.root,text='Crear reporte',command=self.crear_reporte,
                                            justify=CENTER)
        self.boton_crear_reporte.pack(fill='x',expand=True,padx=20,pady=5)
    def crear_guia_despacho(self):
        top = Toplevel(self.root)
        top.geometry('1000x700')
        top.configure(background='dark turquoise')
        top.title("Detalle Compra")
        top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
        
        btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
        btn_volver.pack(side='bottom',padx=10)
        
        self.guia_despachos=pd.merge(left=Ventas['ID_COMPRADOR(RUT)'],right=Despachos,
                                     left_index=True,right_on=['ID_VENTA'])
        
        self.guia_despachos['FECHA_ENTREGA']=pd.to_datetime(self.guia_despachos['FECHA_ENTREGA'])
        self.guia_despachos_filtrada=self.guia_despachos.copy()
        self.guia_despachos_filtrada=self.guia_despachos_filtrada[self.guia_despachos_filtrada['ESTADO']!='ENTREGADO']
        
        frame_opciones=Frame(top)
        frame_opciones.pack(padx=5,pady=6)
        
        frame_guia_despacho=Frame(top)
        frame_guia_despacho.pack(padx=5,pady=6)
        
        frame_exportar_guia=Frame(top)
        frame_exportar_guia.pack(padx=5,pady=5)
        
        self.tabla_guia_despachos = Table(frame_guia_despacho, dataframe=self.guia_despachos_filtrada,height=400,
                                            showtoolbar=True, showstatusbar=True,editable=False,width=700)
        self.tabla_guia_despachos.show()
        
        
        
        label_nombre_guia=Label(frame_exportar_guia,text='Título guía:',justify=CENTER)
        label_nombre_guia.pack(side='left',padx=5)
        
        entry_nombre_guia=Entry(frame_exportar_guia)
        entry_nombre_guia.pack(side='left',padx=5)
        
        def exportar_guia():
            nombre_guia=entry_nombre_guia.get()
            if nombre_guia=='':
                messagebox.showinfo('Error guía despacho','Titulo guía despacho en blanco.')
                return
            
            path_destino=filedialog.askdirectory(title='Selecione ruta de salida')#'Data_test.csv'
            path_destino = str(os.path.normpath(path_destino))+os.sep+nombre_guia+'.csv'
            self.guia_despachos_filtrada.to_csv(path_destino, encoding='utf-8')
            messagebox.showinfo('Guía despacho','Guía despachos generado exitosamente.')
            
           
        
        btn_exportar_guia=Button(frame_exportar_guia,text='Generar guía despacho.',justify=CENTER,command=exportar_guia)
        btn_exportar_guia.pack(side='left',padx=5)
        
        label_rango_fechas=Label(frame_opciones,text='Seleccione rango de fechas:',justify=CENTER)
        label_rango_fechas.pack(side='left',padx=5)
        
        label_fecha_inicial=Label(frame_opciones,text='Fecha inicial:',justify=CENTER)
        label_fecha_inicial.pack(side='left',padx=5)
        entry_fecha_inicial=ttk.Combobox(frame_opciones,state="readonly")
        entry_fecha_inicial['values']=list(np.unique([fecha.date() for fecha in self.guia_despachos['FECHA_ENTREGA']]))
        entry_fecha_inicial.pack(side='left',padx=5)
        
        label_fecha_final=Label(frame_opciones,text='Fecha final:',justify=CENTER)
        label_fecha_final.pack(side='left',padx=5)
        entry_fecha_final=ttk.Combobox(frame_opciones,state="readonly")
        entry_fecha_final['values']=list(np.unique([fecha.date() for fecha in self.guia_despachos['FECHA_ENTREGA']]))
        entry_fecha_final.pack(side='left',padx=5)
        def generar_guia_despacho():
            fecha_inicial=entry_fecha_inicial.get()
            fecha_final=entry_fecha_inicial.get()
            if fecha_inicial=='':
                messagebox.showinfo('Fecha inválida','Seleccione una fecha inicial.')
                return 
            if fecha_final=='':
                messagebox.showinfo('Fecha inválida','Seleccione una fecha final.')
                return 
            fecha_inicial=pd.Timestamp(fecha_inicial)
            fecha_final=pd.Timestamp(fecha_final)
            if fecha_inicial>fecha_final:
                messagebox.showinfo('Fecha inválida','La fecha inicial debe ser menor a la fecha final.')
                return 
            self.guia_despachos_filtrada=self.guia_despachos[self.guia_despachos['FECHA_ENTREGA']>=fecha_inicial]
            self.guia_despachos_filtrada=self.guia_despachos_filtrada[self.guia_despachos['FECHA_ENTREGA']<=fecha_final]
            self.guia_despachos_filtrada=self.guia_despachos_filtrada[self.guia_despachos_filtrada['ESTADO']!='ENTREGADO']
            self.tabla_guia_despachos.tableChanged()
            self.tabla_guia_despachos.destroy()
            self.tabla_guia_despachos = Table(frame_guia_despacho, dataframe=self.guia_despachos_filtrada,height=400,
                                            showtoolbar=True, showstatusbar=True,editable=False,width=700)
            self.tabla_guia_despachos.show()
            messagebox.showinfo('Guía generada exitosamente.','Guía generada exitosamente.')
        btn_gen_guia=Button(frame_opciones,text='Generar guía despacho.',justify=CENTER,command=generar_guia_despacho)
        btn_gen_guia.pack(side='left',padx=5)
        
        
        
        self.root.withdraw()
        
        
        
        
    def ver_detalle_venta(self):
        top = Toplevel(self.root)
        top.geometry('500x500')
        top.configure(background='dark turquoise')
        top.title("Detalle Compra")
        top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
        
        btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
        btn_volver.pack(side='bottom',padx=10)
        
        frame_tabla_ventas=Frame(top)
        frame_tabla_ventas.pack(padx=5)
        tabla_ventas = Table(frame_tabla_ventas, dataframe=Ventas.reset_index(),
                                            showtoolbar=False, showstatusbar=True,editable=False,width=300,height=100)
        tabla_ventas.show()
        
        frame_opciones=Frame(top)
        frame_opciones.pack(padx=5)
        
        label_opciones=Label(frame_opciones,text='ID_VENTA')
        label_opciones.pack(side='left',padx=5)
        
        entry_id_venta=ttk.Combobox(frame_opciones,state="readonly")
        entry_id_venta['values']=list(Ventas.index.values)
        entry_id_venta.pack(side='left',padx=5)
        
        def ver_resumen_aux():
            id_venta=entry_id_venta.get()
            if id_venta=='':
                return
            id_venta=int(id_venta)
            str_lista_venta='Resumen Venta:\n'
            for id_prod in Lista_Ventas[id_venta]:
                prod=Productos.loc[id_prod,'PRODUCTO']
                str_lista_venta=str_lista_venta+prod+': '+str(Lista_Ventas[id_venta][id_prod])+'\n'
            str_lista_venta=str_lista_venta+'Monto Venta: '+str(Ventas.loc[id_venta,'MONTO_PRODUTOS'])+'\n'
            str_lista_venta=str_lista_venta+'Monto Total: '+str(Ventas.loc[id_venta,'MONTO_TOTAL'])+'\n'
            id_despacho=int(Ventas.loc[id_venta,'ID_DESPACHO'])
            str_lista_venta=str_lista_venta+'ID Despacho: '+str(id_despacho)+'\n'
            for col in ['FECHA','CALLE','NUMERO','DEPTO-OTROS','COMUNA','FECHA_ENTREGA']:
                aux=str(Despachos.loc[id_despacho,col])
                str_lista_venta=str_lista_venta+ col+':'+aux+'\n'
            text_resumen.set(str_lista_venta)
        
        btn_ver_resumen=Button(frame_opciones,text='Ver Resumen.',justify=CENTER,command=ver_resumen_aux)
        btn_ver_resumen.pack(side='left',padx=5)
        
        frame_resumen=Frame(top)
        frame_resumen.pack(padx=5)
        text_resumen=StringVar()
        text_resumen.set('Resumen Venta:')
        label_resumen=Label(frame_resumen,textvariable= text_resumen,justify=LEFT)
        label_resumen.pack(padx=5,pady=5)
        
        self.root.withdraw()
    def ver_tablas(self):
        top = Toplevel(self.root)
        top.geometry('1200x600')
        top.configure(background='dark turquoise')
        top.title("Tablas")
        top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
        
        btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
        btn_volver.pack(side='bottom',padx=10)
        
        frame_lables=Frame(top)
        frame_lables.pack(padx=5)
        label_prods=Label(frame_lables,text='Stocks-Productos',justify=CENTER,bg='blue',fg='white',width=50)
        label_prods.pack(fill='x',expand=True,padx=10,pady=5,side='left')
        label_ventas=Label(frame_lables,text='Ventas',justify=CENTER,bg='blue',fg='white',width=50)
        label_ventas.pack(fill='x',expand=True,padx=10,pady=5,side='left')
        label_despachos=Label(frame_lables,text='Despachos',justify=CENTER,bg='blue',fg='white',width=50)
        label_despachos.pack(fill='x',expand=True,padx=10,pady=5,side='left')
        
        frame_productos=Frame(top,width=300,height=100)
        frame_productos.pack(side='left',padx=5,pady=5,fill='x',expand=True)
        
        frame_ventas=Frame(top,width=300,height=100)
        frame_ventas.pack(side='left',padx=5,pady=5,fill='x',expand=True)
        
        frame_despachos=Frame(top,width=300,height=100)
        frame_despachos.pack(side='left',padx=5,pady=5,fill='x',expand=True)
        
        
        tabla_ventas = Table(frame_ventas, dataframe=Ventas.reset_index(),height=400,
                                            showtoolbar=False, showstatusbar=True,editable=False,width=300)
        tabla_ventas.show()
        
        tabla_productos = Table(frame_productos, 
                                dataframe=Productos[['PRODUCTO','CANTIDAD_DISPONIBLE','PRECIO']].reset_index(),
                                           showtoolbar=False,height=400, showstatusbar=True,editable=False,width=300)
        tabla_productos.show()
        
        tabla_despachos = Table(frame_despachos, dataframe=Despachos.reset_index(),
                                            showtoolbar=True,height=400, showstatusbar=True,editable=True,width=300)
        tabla_despachos.show()
        
        self.root.withdraw()
    def eliminar_venta(self):
        top = Toplevel(self.root)
        top.geometry('600x600')
        top.configure(background='dark turquoise')
        top.title("Eliminar Venta")
        top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
        
        btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
        btn_volver.pack(side='bottom',padx=10)
        
        self.Ventas_aux=Ventas.reset_index()
        
        frame_ventas= Frame(top)
        frame_ventas.pack(padx=10,pady=10)
        ventas_anteriores = Table(frame_ventas, dataframe=self.Ventas_aux,
                                            showtoolbar=True, showstatusbar=True,editable=False)
        self.root.withdraw()
        ventas_anteriores.show()
        frame_opciones=Frame(top,width=200)
        frame_opciones.pack(padx=10,pady=10)
        
        label_id_venta=Label(frame_opciones,text='ID_VENTA')
        label_id_venta.pack(padx=5,side='left')
        entry_id_venta=ttk.Combobox(frame_opciones,state="readonly")
        entry_id_venta["values"]=list(Ventas.index.values)
        entry_id_venta.pack(padx=5,side='left')
        def eliminar_venta_aux():
            id_venta=entry_id_venta.get()
            if id_venta=='':
                return
            id_venta=int(id_venta)
            Ventas.drop(labels=id_venta,inplace=True)
            Ventas.to_csv(path_Ventas, encoding='utf-8')
            Despachos.drop(labels=Despachos[Despachos['ID_VENTA']==id_venta].index.values,inplace=True)
            Despachos.to_csv(path_Despachos, encoding='utf-8')
            for prod in Lista_Ventas[id_venta]:
                Productos.loc[prod,'CANTIDAD_DISPONIBLE']=(Productos.loc[prod,'CANTIDAD_DISPONIBLE']+
                                                           Lista_Ventas[id_venta][prod])
                if Productos.loc[prod,'PRODUCTO'] in Ofertas:
                    for prod_aux in Ofertas[Productos.loc[prod,'PRODUCTO']]:
                        id_prod_aux=Productos.loc[Productos['PRODUCTO']==prod_aux].index.values[0]
                        Productos.loc[id_prod_aux,'CANTIDAD_DISPONIBLE']=(Productos.loc[id_prod_aux,'CANTIDAD_DISPONIBLE']+
                                         Lista_Ventas[id_venta][prod]* Ofertas[Productos.loc[prod,'PRODUCTO']][prod_aux])
            Productos.to_csv(path_Productos, encoding='utf-8')
            Lista_Ventas.pop(id_venta)
            with open(path_Lista_Ventas, 'wb') as file:
                pickle.dump(Lista_Ventas, file)
            self.Ventas_aux.drop(self.Ventas_aux.index[self.Ventas_aux['ID_VENTA']==id_venta],inplace=True)
            ventas_anteriores.tableChanged()
        btn_eliminar_venta=Button(frame_opciones,text='Eliminar.',command=eliminar_venta_aux)
        
        btn_eliminar_venta.pack(padx=5,side='left')
        #entry_id_venta
        
    def nueva_venta(self):
        fecha=pd.Timestamp(datetime.now())
        fecha=fecha.replace(second=0,microsecond=0,nanosecond=0,minute=0,hour=0)
        self.lista_venta_acutal={}
        top = Toplevel(self.root)
        top.geometry('1200x600')
        top.configure(background='dark turquoise')
        top.title("Nueva Venta")
        top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
        
        btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
        btn_volver.pack(side='bottom',padx=10)
        
        def crear_venta_aux():
            lista_venta_acutal_por_ids={}
            if self.lista_venta_acutal:
                
                for prod in self.lista_venta_acutal:
                    id_prod=Productos[Productos['PRODUCTO']==prod].index.values[0]
                    lista_venta_acutal_por_ids[id_prod]=int(self.lista_venta_acutal[prod])
            else:
                messagebox.showinfo('Lista productos inválida.','Debe seleccionar al menos un producto.')
                return
            descuento=entry_descuento.get()
            if descuento.isdigit():
                descuento=int(descuento)
            else:
                messagebox.showinfo('Descuento invalido.','El descuento debe ser un número positivo.')
                return
            telefono=entry_tel.get()
            id_comprador=entry_comprador.get()
            fecha_entrega=entry_fecha_entrega.get()
            dept=entry_depto.get()
            comuna=entry_comuna.get()
            numero=entry_numero.get()
            monto_despacho=entry_monto_despacho.get()
            if monto_despacho.isdigit():
                monto_despacho=int(monto_despacho)
            else:
                messagebox.showinfo('Monto despacho invalido.','El monto de despacho debe ser un número positivo.')
                return
            calle=entry_calle.get()
            id_venta=crear_venta(id_comprador,telefono,lista_venta_acutal_por_ids,str(fecha.date()),
            calle=calle,numero=numero,depto=dept,comuna=comuna,fecha_entrega=fecha_entrega,
               monto_despacho=monto_despacho,descuento=descuento)
            
            entry_descuento.delete(0,len(entry_descuento.get()))
            entry_comprador.delete(0,len(id_comprador))
            entry_calle.delete(0,len(calle))
            entry_numero.delete(0,len(numero))
            entry_depto.delete(0,len(dept))
            entry_comuna.delete(0,len(comuna))
            entry_cantidad.delete(0,len(entry_cantidad.get()))
            entry_monto_despacho.delete(0,len(entry_monto_despacho.get()))
            
            id_venta=int(id_venta)
            str_lista_venta='Resumen Venta:\n'
            for id_prod in Lista_Ventas[id_venta]:
                prod=Productos.loc[id_prod,'PRODUCTO']
                str_lista_venta=str_lista_venta+prod+': '+str(Lista_Ventas[id_venta][id_prod])+'\n'
            str_lista_venta=str_lista_venta+'Monto Venta: '+str(Ventas.loc[id_venta,'MONTO_PRODUTOS'])+'\n'
            str_lista_venta=str_lista_venta+'Monto Total: '+str(Ventas.loc[id_venta,'MONTO_TOTAL'])+'\n'
            id_despacho=int(Ventas.loc[id_venta,'ID_DESPACHO'])
            str_lista_venta=str_lista_venta+'ID Despacho: '+str(id_despacho)+'\n'
            for col in ['FECHA','CALLE','NUMERO','DEPTO-OTROS','COMUNA','FECHA_ENTREGA']:
                aux=str(Despachos.loc[id_despacho,col])
                str_lista_venta=str_lista_venta+ col+':'+aux+'\n'

            texto_lista_prods.set('Venta Generada exitosamente.\n'+str_lista_venta)
            self.lista_venta_acutal={}
        btn_crear_venta=Button(top,command=crear_venta_aux,text='Crear Venta.')
        btn_crear_venta.pack(side='bottom',padx=10)
        
        texto_lista_prods=StringVar()
        texto_lista_prods.set('Lista de Producto:\n')
        label_lista_prods=Label(top, textvariable=texto_lista_prods,justify=CENTER,bg='blue',fg='white')
        label_lista_prods.pack(padx=10,pady=10,fill='x',expand=True)
        
        frame1=Frame(top,bg='dark turquoise')
        frame1.pack(fill='x',expand=True,padx=10,pady=10)
        
        label_comprador= Label(frame1,text='Nombre o Rut Comprador:')
        label_comprador.pack(side='left',padx=10)
        entry_comprador= Entry(frame1,width=30)
        entry_comprador.pack(side='left',padx=10)
        
        label_tel= Label(frame1,text='Número teléfono:')
        label_tel.pack(side='left',padx=10)
        entry_tel= Entry(frame1,width=30)
        entry_tel.pack(side='left',padx=10)
        
        label_fecha_entrega= Label(frame1,text='Fecha entrega:')
        label_fecha_entrega.pack(side='left')
        entry_fecha_entrega= ttk.Combobox(frame1,state="readonly")
        entry_fecha_entrega["values"]=list([str(fecha_aux.date()) 
                                            for fecha_aux in pd.date_range(fecha,periods=5,freq='D')])
        entry_fecha_entrega.pack(side='left',padx=10)
        
        label_descuento= Label(frame1,text='Descuento',justify=CENTER)
        label_descuento.pack(side='left',padx=10)
        entry_descuento=Entry(frame1,text='0')
        entry_descuento.pack(side='left',padx=10)
        
        
        frame2=Frame(top,bg='dark turquoise')
        frame2.pack(fill='x',expand=True,padx=10,pady=10)
        
        #dept=depto_entry.get()
        #    comuna=comuna_entry.get()
        #    numero=numero_entry.get()
        #    monto_despacho=monto_despacho_entry.get()
        #    calle=calle_entry.get()
        
        
        label_calle=Label(frame2,text='Calle:')
        label_calle.pack(side='left',padx=10)
        entry_calle=Entry(frame2)
        entry_calle.pack(side='left',padx=10)
        
        label_numero=Label(frame2,text='Número:')
        label_numero.pack(side='left',padx=10)
        entry_numero=Entry(frame2)
        entry_numero.pack(side='left',padx=10)
        
        label_depto=Label(frame2,text='Depto:')
        label_depto.pack(side='left',padx=10)
        entry_depto=Entry(frame2)
        entry_depto.pack(side='left',padx=10)
        
        label_comuna=Label(frame2,text='Comuna:')
        label_comuna.pack(side='left',padx=10)
        entry_comuna=Entry(frame2)
        entry_comuna.pack(side='left',padx=10)
        
        label_monto_despacho=Label(frame2,text='Monto Despacho:')
        label_monto_despacho.pack(side='left',padx=10)
        entry_monto_despacho=Entry(frame2)
        entry_monto_despacho.pack(side='left',padx=10)
        
        
        frame3=Frame(top,bg='dark turquoise')
        frame3.pack(fill='x',expand=True,padx=10,pady=10)
        
        
        label_sel_prod= Label(frame3,text='Seleccione producto:')
        label_sel_prod.pack(side='left',padx=10)
        combo1 = ttk.Combobox(frame3, state="readonly",width=100)
        combo1["values"] = list(Productos['PRODUCTO'].unique())
        combo1.pack(side='left',padx=10)
        
        label_cantidad= Label(frame3,text='Canitdad:')
        label_cantidad.pack(side='left')
        entry_cantidad= Entry(frame3,width=10)
        entry_cantidad.pack(side='left',padx=10)
        
        
        
        
        
        def agregar_prod_lista_venta():
            producto=combo1.get()
            cantidad=entry_cantidad.get()
            if producto=='':
                messagebox.showinfo('Error producto','Debe seleccionar el producto.')
                return
            if cantidad.isdigit():
                cantidad=int(cantidad)
                if cantidad<=0:
                    messagebox.showinfo('Error cantidad','Debe seleccionar al menos una unidad.')
                    return
            else:
                messagebox.showinfo('Error cantidad','Debe ingresar la cantidad.')
                return
            if Productos.loc[Productos['PRODUCTO']==producto,'CANTIDAD_DISPONIBLE'].values[0]-cantidad<0:
                messagebox.showinfo('Producto sin stock suficiente.','Producto '+producto+' sin stock suficiente.')
                return
            self.lista_venta_acutal[producto]=cantidad
            str_lista_venta_actual='Lista de Producto:\n'
            for prod in self.lista_venta_acutal:
                str_lista_venta_actual=str_lista_venta_actual+prod+': '+str(self.lista_venta_acutal[prod])+'\n'
            
            
            texto_lista_prods.set(str_lista_venta_actual)
            
        btn_add_prod= Button(frame3,text='Agregar a la lista.',command=agregar_prod_lista_venta,justify=CENTER)
        btn_add_prod.pack(side='left',padx=10)
        
        
        self.root.withdraw()
    def gestionar_prods(self):
        top = Toplevel(self.root)
        top.geometry('1000x800')
        top.configure(background='dark turquoise')
        top.title("Gestión de Productos")
        top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
        
        btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
        btn_volver.pack(side='bottom',expand=True,fill='x',padx=10)
        
        frame_tabla_prods=Frame(top)
        frame_tabla_prods.pack(padx=10,pady=10)
        tabla_productos = Table(frame_tabla_prods, dataframe=Productos,
                                           showtoolbar=False, showstatusbar=True,editable=False,width=300,height=100)
        tabla_productos.show()
        
        frame_agregar_prod=Frame(top)
        frame_agregar_prod.pack(padx=10,pady=5,expand=True,fill='x')
        
        frame_modificar_prod=Frame(top)
        frame_modificar_prod.pack(padx=10,pady=5,expand=True,fill='x')
        
        frame_agregar_oferta=Frame(top)
        frame_agregar_oferta.pack(padx=10,pady=5,expand=True,fill='x')

        
        texto_resultado=StringVar()
        texto_resultado.set('Agregue un nuevo producto:')
        label_resultado=Label(frame_agregar_prod,textvariable=texto_resultado,justify=CENTER)
        label_resultado.pack()
        def agregar_producto():
            producto=entry_nomb_prod.get()
            descripcion=entry_descripcion.get()
            cantidad=entry_cantidad.get()
            precio=entry_precio.get()

            if producto=='':
                texto_resultado.set('Nombre del producto invalido.')
                return 
            if cantidad.isdigit()==False:
                texto_resultado.set('La cantidad debe ser un número entero.')
                return 
            cantidad=int(cantidad)
            if precio.isdigit()==False:
                texto_resultado.set('El precio debe ser un número entero.')
                return 
            precio=int(precio)
            crear_producto(producto,descripcion,cantidad,precio)
            texto_resultado.set('Producto agregado exitosamente.')
        btn_agregar_prod=Button(frame_agregar_prod,text='Agregar Producto',command=agregar_producto,width=100)
        btn_agregar_prod.pack(side='bottom',fill='x',padx=10)
        
        label_nomb_prod=Label(frame_agregar_prod,text='Nombre Producto',justify=CENTER)
        label_nomb_prod.pack(side='left',padx=10,pady=10)
        entry_nomb_prod=Entry(frame_agregar_prod)
        entry_nomb_prod.pack(side='left',padx=10)
        
        label_descripcion=Label(frame_agregar_prod,text='Descripción')
        label_descripcion.pack(side='left',padx=10)
        entry_descripcion=Entry(frame_agregar_prod)
        entry_descripcion.pack(side='left',padx=10)
        
        label_precio=Label(frame_agregar_prod,text='Precio')
        label_precio.pack(side='left')
        entry_precio=Entry(frame_agregar_prod,width=10,text='precio')
        entry_precio.pack(side='left',padx=10)
        
        label_cantidad=Label(frame_agregar_prod,text='Cantidad')
        label_cantidad.pack(side='left')
        entry_cantidad=Entry(frame_agregar_prod,width=10,text='cantidad')
        entry_cantidad.pack(side='left',padx=10)
        
        self.root.withdraw()
        
        
        ####Para modificar un producto
        productos_menos_ofertas= list([ prod for prod in Productos['PRODUCTO'] if prod not in Ofertas])
        
        label_title_mod=Label(frame_modificar_prod,text='Modificación Productos',justify=CENTER)
        label_title_mod.pack(padx=5,pady=10)
        label_producto_mod=Label(frame_modificar_prod,text='Producto:',justify=CENTER)
        label_producto_mod.pack(side='left',padx=5)
        
        entry_producto_mod=ttk.Combobox(frame_modificar_prod,state="readonly")
        entry_producto_mod['values']=productos_menos_ofertas
        entry_producto_mod.pack(side='left',padx=5)
        
        label_precio_mod=Label(frame_modificar_prod,text='Precio nuevo:',justify=CENTER)
        label_precio_mod.pack(side='left',padx=5)
        entry_precio_mod=Entry(frame_modificar_prod)
        entry_precio_mod.pack(side='left',padx=5)
        
        label_cantidad_mod=Label(frame_modificar_prod,text='Cantidad nueva:',justify=CENTER)
        label_cantidad_mod.pack(side='left',padx=5)
        entry_cantidad_mod=Entry(frame_modificar_prod)
        entry_cantidad_mod.pack(side='left',padx=5)
        
        def modificar_prod_aux():
            producto=entry_producto_mod.get()
            if producto=='':
                return
            precio=entry_precio_mod.get()
            cantidad=entry_cantidad_mod.get()
            if precio.isdigit():
                precio=int(precio)
            elif precio=='':
                precio=None
            else:
                return
            if cantidad.isdigit():
                cantidad=int(cantidad)
            elif cantidad=='':
                cantidad=None
            else:
                return
            if precio is None and cantidad is None:
                return 
            id_product=Productos.loc[Productos['PRODUCTO']==producto].index.values[0]
            modificar_producto(id_product,cantidad,precio)
            tabla_productos.tableChanged()
            
        btn_mod=Button(frame_modificar_prod,text='Modificar Producto.',command=modificar_prod_aux,justify=CENTER)
        btn_mod.pack(side='left',padx=5)
        
        
        self.lista_productos_oferta={}
        
        
        def agregar_oferta_aux():
            if self.lista_productos_oferta:
                nombre_oferta=entry_nombre_oferta.get()
                precio=entry_precio_oferta.get()
                if nombre_oferta=='':
                    return
                if precio.isdigit():
                    precio=int(precio)
                else:
                    return
                crear_oferta(nombre_oferta,self.lista_productos_oferta,precio)

                txt_oferta.set('Oferta agregada exitosamente.')
                tabla_productos.tableChanged()
                self.lista_productos_oferta={}
        btn_agregar_oferta=Button(frame_agregar_oferta,text='Agregar Oferta',justify=CENTER,
                                  command=agregar_oferta_aux)
        btn_agregar_oferta.pack(padx=5,expand=True,fill='x',side='bottom')
        
        txt_oferta= StringVar()
        txt_oferta.set('Resumen Oferta:')
        label_detalle_oferta=Label(frame_agregar_oferta,textvariable=txt_oferta,justify=CENTER)
        label_detalle_oferta.pack(padx=5,pady=5,expand=True,fill='x',side='bottom')
        
        label_agregar_oferta= Label(frame_agregar_oferta,text='Agregar Oferta',justify=LEFT)
        label_agregar_oferta.pack(padx=5)
        
        label_nombre_oferta= Label(frame_agregar_oferta,text='Nombre Oferta',justify=LEFT)
        label_nombre_oferta.pack(side='left',padx=5)
        entry_nombre_oferta =Entry(frame_agregar_oferta)
        entry_nombre_oferta.pack(side='left',padx=5)
        
        label_prod_oferta=Label(frame_agregar_oferta,text='Producto:')
        label_prod_oferta.pack(side='left',padx=5)
        entry_prod_oferta=ttk.Combobox(frame_agregar_oferta,state="readonly")
        entry_prod_oferta['values']=productos_menos_ofertas
        entry_prod_oferta.pack(side='left',padx=5)
        
        label_cantidad_oferta=Label(frame_agregar_oferta,text='Cantidad:')
        label_cantidad_oferta.pack(side='left',padx=5)
        entry_cantidad_oferta=Entry(frame_agregar_oferta)
        entry_cantidad_oferta.pack(side='left',padx=5)
        
        label_precio_oferta=Label(frame_agregar_oferta,text='Precio:')
        label_precio_oferta.pack(side='left',padx=5)
        entry_precio_oferta=Entry(frame_agregar_oferta)
        entry_precio_oferta.pack(side='left',padx=5)
        def agregar_prod_oferta():
            
            prod=entry_prod_oferta.get()
            cantidad=entry_cantidad_oferta.get()
            if prod=='':
                return
            if cantidad.isdigit():
                cantidad=int(cantidad)
            else:
                return
            self.lista_productos_oferta[prod]=cantidad
            str_txt_oferta='Resumen Oferta:\n'
            for prod in self.lista_productos_oferta:
                str_txt_oferta=str_txt_oferta+prod+':'+str(self.lista_productos_oferta[prod])+'\n'
            txt_oferta.set(str_txt_oferta)
        
                
        btn_agregar_prod_oferta=Button(frame_agregar_oferta,text='Agregar producto.',command=agregar_prod_oferta)
        btn_agregar_prod_oferta.pack(side='left',padx=5)
        self.root.withdraw()
    def crear_reporte(self):
        pass
    def mostrar_csv(self):
        if self.data is not None:
            top = Toplevel(self.root)
            top.configure(background='dark turquoise')
            top.title("Leyendo "+self.ruta_data)
        
            f=Frame(top)
            f.grid(row=1,column=0,columnspan=2)
            tabla = pt = Table(f, dataframe=self.data,
                                            showtoolbar=True, showstatusbar=True)
            top.protocol("WM_DELETE_WINDOW", functools.partial(self.volver, self.root,top))
            btn_volver = Button(top,text="Volver", command=functools.partial(self.volver, self.root,top))
            btn_volver.grid(row=0,column=0,columnspan=2)
        
            self.root.withdraw()
        
            pt.show()
        else:
            self.text_archivo.set('Debe cargar el arhivo primero.')
            return -1
    
    def volver(self, root,top):
        root.deiconify()
        top.destroy()
programa=Programa()
programa.root.mainloop()
Productos.to_csv(path_Productos, encoding='utf-8')
Ventas.to_csv(path_Ventas, encoding='utf-8')
Despachos.to_csv(path_Despachos, encoding='utf-8')
with open(path_Ofertas, 'wb') as file:
    pickle.dump(Ofertas, file)
with open(path_Lista_Ventas, 'wb') as file:
    pickle.dump(Lista_Ventas, file)